# FixedIncomeLib SABR Model Testing Notebook
This notebook exercises the `SabrModel` class by:
1. Building a dummy SABR volatility surface for a single index.
2. Interpolating volatilities at various expiry/tenor grid points.
---

In [1]:
import pandas as pd
from fixedincomelib.date import Date, Period
from fixedincomelib.market import IndexRegistry
import numpy as np
from fixedincomelib.data import Data1D, Data2D, DataCollection
from fixedincomelib.sabr import SabrModel
from fixedincomelib.yield_curve import YieldCurve

2. Dummy Yield Curve for subModel (flat rates)
We build a simple flat curve to satisfy SabrModel's subModel requirement.

In [2]:
DUMMY_YC = [
    ["SOFR-1B",            "1M",  0.0020],
    ["SOFR-1B",            "3M",  0.0025],
    ["SOFR-1B",            "6M",  0.0030],
    ["USD-LIBOR-BBA-3M",   "1M",  0.0040],
    ["USD-LIBOR-BBA-3M",   "3M",  0.00425],
    ["USD-LIBOR-BBA-3M",   "6M",  0.00450],
]
df_yc = pd.DataFrame(DUMMY_YC, columns=["INDEX","AXIS1","VALUES"])

yc_build = [
  {
    "TARGET":             "SOFR-1B",
    "DATA_TYPE":          "ZERO_RATE",
    "DATA_CONVENTION":    "SOFR-1B",
    "INTERPOLATION_METHOD":"PIECEWISE_CONSTANT"
  },
  {
    "TARGET":             "USD-LIBOR-BBA-3M",
    "DATA_TYPE":          "ZERO_RATE",
    "DATA_CONVENTION":    "USD-LIBOR-BBA-3M",
    "INTERPOLATION_METHOD":"PIECEWISE_CONSTANT"
  },
]



In [3]:
data_objs = []
for idx_name, sub in df_yc.groupby("INDEX"):
    d1 = Data1D.createDataObject(
        data_type      = "zero_rate",
        data_convention= idx_name,
        df             = sub[["AXIS1","VALUES"]]
    )
    data_objs.append(d1)

dc = DataCollection(data_objs)
print("Registered zero-rate keys:", dc.dataMap.keys())

Registered zero-rate keys: dict_keys([('zero_rate', 'SOFR-1B'), ('zero_rate', 'USD-LIBOR-BBA-3M')])


In [4]:
as_of = "2025-06-26"
yc = YieldCurve(as_of, dc, yc_build)
print("YieldCurve components:", yc.components.keys())


ValueError: build_method must include a non-empty 'INSTRUMENTS' list.

3. Dummy SABR Data
Columns: INDEX, EXPIRY (years), TENOR (years), NORMALVOL, BETA, NU, RHO

In [ ]:
DUMMY_SABR = [
  ["USD-LIBOR-BBA-3M", 1.0, 0.25, 0.010, 0.5, 0.2, -0.3],
  ["USD-LIBOR-BBA-3M", 1.0, 0.50, 0.011, 0.5, 0.2, -0.3],
  ["USD-LIBOR-BBA-3M", 2.0, 0.25, 0.012, 0.5, 0.2, -0.3],
  ["USD-LIBOR-BBA-3M", 2.0, 0.50, 0.013, 0.5, 0.2, -0.3],
]
df_sabr = pd.DataFrame(
    DUMMY_SABR,
    columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"]
)

for idx_name, sub in df_sabr.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0)
            .sort_index(axis=1)
        )
        d2 = Data2D.createDataObject(
            data_type       = param.lower(),
            data_convention = idx_name,
            df              = pivot
        )
        data_objs.append(d2)

dc = DataCollection(data_objs)
print("Registered SABR surface keys:", dc.dataMap.keys())


Registered SABR surface keys: dict_keys([('zero_rate', 'SOFR-1B'), ('zero_rate', 'USD-LIBOR-BBA-3M'), ('normalvol', 'USD-LIBOR-BBA-3M'), ('beta', 'USD-LIBOR-BBA-3M'), ('nu', 'USD-LIBOR-BBA-3M'), ('rho', 'USD-LIBOR-BBA-3M')])


In [ ]:
build_methods_sabr = [
    {
        "TARGET":        "USD-LIBOR-BBA-3M",
        "VALUES":        "NORMALVOL",
        "INTERPOLATION": "LINEAR",
    },
    {
        "TARGET":        "USD-LIBOR-BBA-3M",
        "VALUES":        "BETA",
        "INTERPOLATION": "LINEAR",
    },
    {
        "TARGET":        "USD-LIBOR-BBA-3M",
        "VALUES":        "NU",
        "INTERPOLATION": "LINEAR",
    },
    {
        "TARGET":        "USD-LIBOR-BBA-3M",
        "VALUES":        "RHO",
        "INTERPOLATION": "LINEAR",
    },
]


In [ ]:
value_date = "2025-06-26"
sabr = SabrModel.from_curve(
    valueDate              = value_date,
    dataCollection         = dc,
    buildMethodCollection  = build_methods_sabr,
    ycModel                = yc
)

print("SABR components:", sabr.components.keys())

SABR components: dict_keys(['USD-LIBOR-BBA-3M-NORMALVOL', 'USD-LIBOR-BBA-3M-BETA', 'USD-LIBOR-BBA-3M-NU', 'USD-LIBOR-BBA-3M-RHO'])


In [ ]:
grid = [(1.0, 0.25), (1.0, 0.50), (2.0, 0.25), (2.0, 0.50)]
print("--- SABR Volatility Interpolation ---")
for expiry, tenor in grid:
    normalvol, beta, nu, rho, shift, decay = sabr.get_sabr_parameters(
        "USD-LIBOR-BBA-3M",
        expiry,
        tenor
    )
    print(f"Expiry={expiry}y, Tenor={tenor}y → Vol = {normalvol:.4%}")


--- SABR Volatility Interpolation ---
Expiry=1.0y, Tenor=0.25y → Vol = 1.0000%
Expiry=1.0y, Tenor=0.5y → Vol = 1.1000%
Expiry=2.0y, Tenor=0.25y → Vol = 1.2000%
Expiry=2.0y, Tenor=0.5y → Vol = 1.3000%


In [ ]:
print("--- Off-Grid SABR Volatility Interpolation ---")
off_grid = [
    (1.5, 0.25),
    (1.0, 0.375),
    (1.5, 0.375)
]
target = "USD-LIBOR-BBA-3M"
for expiry, tenor in off_grid:
    normalvol, beta, nu, rho, shift, decay = sabr.get_sabr_parameters(target, expiry, tenor)
    print(f"Expiry={expiry:>4.3f}y, Tenor={tenor:>5.3f}y → Vol = {normalvol:.4%}")

--- Off-Grid SABR Volatility Interpolation ---
Expiry=1.500y, Tenor=0.250y → Vol = 1.1000%
Expiry=1.000y, Tenor=0.375y → Vol = 1.0500%
Expiry=1.500y, Tenor=0.375y → Vol = 1.1500%


In [ ]:
print("=== SABR Model via from_curve (prebuilt YieldCurve) ===")

sabr_a = SabrModel.from_curve(
    valueDate             = value_date,           
    dataCollection        = dc,                   
    buildMethodCollection = build_methods_sabr,  
    ycModel               = yc                  
)

print("SABR components:", sabr_a.components.keys())
for expiry, tenor in [(1.5,0.375), (2.0,0.5)]:
    normalvol, beta, nu, rho, shift, decay = sabr_a.get_sabr_parameters(
        "USD-LIBOR-BBA-3M", expiry, tenor
    )
    print(f"[from_curve] Expiry={expiry}, Tenor={tenor} → Vol={normalvol:.4%}")

print("\n=== SABR Model via from_data (raw YC data & build methods) ===")

sabr_b = SabrModel.from_data(
    valueDate             = value_date,           
    dataCollection        = dc,                   
    buildMethodCollection = build_methods_sabr,  
    ycData                 = df_yc,               
    ycBuildMethods         = yc_build             
)
print("SABR components:", sabr_b.components.keys())
for expiry, tenor in [(1.5,0.375), (2.0,0.5)]:
    normalvol, beta, nu, rho, shift, decay = sabr_b.get_sabr_parameters(
        "USD-LIBOR-BBA-3M", expiry, tenor
    )
    print(f"[from_data ] Expiry={expiry}, Tenor={tenor} → Vol={normalvol:.4%}")


=== SABR Model via from_curve (prebuilt YieldCurve) ===
SABR components: dict_keys(['USD-LIBOR-BBA-3M-NORMALVOL', 'USD-LIBOR-BBA-3M-BETA', 'USD-LIBOR-BBA-3M-NU', 'USD-LIBOR-BBA-3M-RHO'])
[from_curve] Expiry=1.5, Tenor=0.375 → Vol=1.1500%
[from_curve] Expiry=2.0, Tenor=0.5 → Vol=1.3000%

=== SABR Model via from_data (raw YC data & build methods) ===
SABR components: dict_keys(['USD-LIBOR-BBA-3M-NORMALVOL', 'USD-LIBOR-BBA-3M-BETA', 'USD-LIBOR-BBA-3M-NU', 'USD-LIBOR-BBA-3M-RHO'])
[from_data ] Expiry=1.5, Tenor=0.375 → Vol=1.1500%
[from_data ] Expiry=2.0, Tenor=0.5 → Vol=1.3000%
